Note book to explore solutions.

Clean and remove null cells

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/movies_data_1.csv")

Data cleaning

In [ ]:

#get data from file data

# DATA CLEANING

#drop null columns
allColumns = ['MOVIES','YEAR','GENRE','RATING','ONE-LINE','STARS','VOTES','RunTime','Gross']


#df[allColumns] = df[allColumns].replace(['', ' ', 'nan', 'NaN'], pd.NA)

#df = df.dropna(subset=allColumns, how='all')
    #drop after
#df.dropna(subset=allColumns, inplace=True)
df = df.dropna(subset=allColumns, axis=0, how='all')
#df[allColumns] = df[allColumns].replace(r'^\s*$', np.nan, regex=True)



#select only some columns
nColumns = ['GENRE','ONE-LINE','STARS']
ownerColumn = ['owner_company']

#clean where you find \n and ''space
for col in nColumns:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('\n', '', regex=False)
            .str.strip()
        )
# to clean only in owner column
for col in ownerColumn:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace('\t', '', regex=False)
            .str.strip()
        )
    #select all columns

print(df.head(25))

Column spliting

In [ ]:
#COLUMN SPLITING

print(df['STARS'].head(10))

#strip column names
#df.columns = df.columns.str.strip()

#fill empty rows
df['STARS'] = df['STARS'].fillna('')

has_pipe = df['STARS'].str.contains('|', regex=False)

#split stars to director and starslist
df[['DIRECTOR','STARS-LIST']] = df.loc[has_pipe, 'STARS'].str.split('|', n=1, expand=True)
df.loc[~has_pipe, 'DIRECTOR'] = ''
df.loc[~has_pipe, 'STARS-LIST'] = df.loc[~has_pipe, 'STARS']
#df['STARS'] =df['STARS'].fillna('|')

#clean whitespace with str.strip()
#df['DIRECTOR'] = df['STARS'].str.extract(r'Director: (.*)')
#df['STARS-LIST'] = df['STARS'].str.extract(r'Stars: (.*)')

df['DIRECTOR'] = df['DIRECTOR'].str.strip()
df['STARS-LIST'] = df['STARS-LIST'].str.strip()

#df['STARS-LIST'] = df['STARS-LIST'].replace('', np.nan)
print(df['DIRECTOR'].head(10))
print(df['STARS-LIST'].head(10))

df.drop(columns=['STARS'], inplace=True)



In [ ]:
print(df.head(10))

Convert to date object

In [ ]:
df['Extract_date'] = pd.to_datetime(df['Extract_date'],errors='coerce') #to_datetime converts it to a object

#create column from extact date and time
df['extraction_date'] = df['Extract_date'].dt.date
df['extraction_time'] = df['Extract_date'].dt.time

print(df[['extraction_time','extraction_date']].head())

df.drop(columns=['Extract_date'], inplace=True)


Year calculations function

In [ ]:
#YEAR CALCULATIONS

#convert to string
#df['YEAR'] = df['YEAR'].astype(str)

#to fix index a
df.reset_index(drop=True, inplace=True)

df['YEAR'] = df['YEAR'].fillna('')

df['YEAR'] = df['YEAR'].str.replace(r'\(I\)', '', regex=True)
df['YEAR'] = df['YEAR'].str.replace(r'\(II\)', '', regex=True)
df['YEAR'] = df['YEAR'].str.replace(r'\(III\)', '', regex=True)

df['YEAR'] = df['YEAR'].str.replace(r'\(', '', regex=True)
df['YEAR'] = df['YEAR'].str.replace(r'\)', '', regex=True)

df['YEAR'] = df['YEAR'].str.replace('–', '-')

df['YEAR'] = df['YEAR'].str.replace(r'[^\d-]', '', regex=True)

df['YEAR'] = df['YEAR'].str.strip()

df['YEAR'] = df['YEAR'].replace('', '-')

df[['start_year','end_year']] = df['YEAR'].str.split('-',n=1, expand=True)

df['end_year'] = df['end_year'].fillna(df['start_year'])

df['end_year'] = df['end_year'].replace('', 'present')

#run a funciton with .apply to each row(axis=1)
#df['duration_years'] = df.apply(lambda row: int(row['end_year']) - int(row['start_year']) + 1 if row['end_year'] != 'present' else pd.NA, axis=1)

durations = []

for i in range(len(df)):
    start = df.loc[i,'start_year']
    end = df.loc[i,'end_year']

    if end == 'present':
        durations.append(pd.NA)
    else:
        durations.append(int(end) - int(start) + 1) #so it also gets months

df['duration_in_years'] = durations

df.drop(columns=['YEAR'], inplace=True)




Create different dataframes

In [ ]:
#dimcompany is probably owner_company
#dimdirector is probably director

unique_companies = df['owner_company'].dropna().unique()

DimCompan = pd.DataFrame(unique_companies, columns=['DimCompan'])
print(DimCompan.head())


unique_directors = df['DIRECTOR'].dropna().unique()

DimDirector = pd.DataFrame(unique_directors, columns=['DimDirector'])
print(DimDirector.head())

DimCompan.to_csv(f"../output/uniqueCompany.csv",index=False)
DimDirector.to_csv(f"../output/uniqueDirector.csv",index=False)


In [ ]:
print(df.head(10))

In [ ]:
#to check if data is changed correctly

from datetime import datetime

timeStamp = datetime.now().strftime("%H%M%S")

#df.to_csv(f"../output/secondOutputAtTime{timeStamp}.csv",index=False)
#df.to_csv(f"../output/outputDataCleaned.csv",index=False)
#df.to_csv(f"../output/outputAtTime{timeStamp}.csv",index=False)
df.to_excel("../output/finalOutput2.xlsx",index=False)